### __Migraciones__

***

In [386]:
import pandas as pd

Carga del dataset de Migraciones.

In [387]:
Dataset_Migraciones = 'Migraciones'
Ruta = ''
Migraciones_DB = pd.read_csv(Ruta + Dataset_Migraciones + '.csv')

Comienzo del filtrado. Lo primero es quedarnos con las filas de "TOT" en la columna de 'Migration by Gender Code'. Las filas FEM y MAL, en conjunto y sumadas, dan TOT, por ende sería información repetida e innecesaria para los fines del objetivo del estudio.

In [388]:
Migrantes = Migraciones_DB[Migraciones_DB['Migration by Gender Code'] == 'TOT']

Eliminaamos las columnas que no incluyen a Argentina como país de origen o destino.

In [389]:
Migrantes = Migrantes[(Migrantes['Country Origin Code'] == 'ARG') | 
                      (Migrantes['Country Dest Code'] == 'ARG')]

Filtramos las columnas a las que nos importan. Además, reseteamos el índice para no generar problemas.

In [390]:
Columnas = ['Country Origin Code', 'Country Dest Code', 
            '1960 [1960]', '1970 [1970]', '1980 [1980]', '1990 [1990]', '2000 [2000]']

Migrantes = Migrantes[Columnas]
Migrantes = Migrantes.reset_index(drop=True)

Cambiamos los nombres de los atributos a los definitivos.

In [391]:
Nuevos_Nombres = {'Country Origin Code': 'id_pais_origen',
                  'Country Dest Code': 'id_pais_destino',
                  '1960 [1960]': 'cantidad_en_1960',
                  '1970 [1970]': 'cantidad_en_1970',
                  '1980 [1980]': 'cantidad_en_1980',
                  '1990 [1990]': 'cantidad_en_1990',
                  '2000 [2000]': 'cantidad_en_2000'}

Migrantes = Migrantes.rename(columns=Nuevos_Nombres)

Ya que las columnas de "cantidad_en_x" deben tener enteros, si hay strings, los convertimos en NULLS.

In [392]:
Columnas_Numericas = ['cantidad_en_1960', 'cantidad_en_1970', 'cantidad_en_1980', 
                      'cantidad_en_1990', 'cantidad_en_2000']

for Columna in Columnas_Numericas:
    Migrantes[Columna] = pd.to_numeric(Migrantes[Columna], errors='coerce')

Verificamos si hay NULLS en las columnas.

In [393]:
Count_Nulls_By_Columns = Migrantes.isnull().sum()

Llenamos los NULLS con 0. Esto es porque en la tabla había "." para cuando no había habido migraciones en ninguna de las columnas numéricas, por lo que 0 es un relleno correcto.

In [394]:
for Columna in Columnas_Numericas:
    Migrantes[Columna] = Migrantes[Columna].fillna(0)

Por las dudas, borramos espacios entre palabras (que suponemos no los hay, pero por si los hubiere perdidos por ahí) y al principio y al final.

In [395]:
Columnas_String = ['id_pais_origen', 'id_pais_destino']

for Columna in Columnas_String:
    Migrantes[Columna] = Migrantes[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)

Creamos cinco DataFrames, cada uno con un año específico. Le agregamos una columna con el año correspondiente. Después, los filtramos: nos quedamos solo con las filas con valores mayores a 0, es decir, con filas con migrantes. Reseteamos el índice y les cambiamos el nombre. Por último, los concatenamos en el DataFrame final.

In [396]:
DataFrames_Cantidades = []

for Columna in Columnas_Numericas:
    DataFrame = Migrantes[['id_pais_origen', 'id_pais_destino', Columna]]    # Filtrado de columnas.
    DataFrame = DataFrame[DataFrame[Columna] > 0]                            # Filtrado de filas.
    DataFrame['anio'] = Columna[-4:len(Columna)+1]                           # Crear columna con el año.    
    DataFrame = DataFrame.reset_index(drop=True)                             # Resetear índices.
    DataFrame = DataFrame.rename(columns={Columna: 'migrantes'})             # Renombrar columna.
    DataFrames_Cantidades.append(DataFrame)                                  # Agregar df a lista de dfs.

Migrantes = pd.concat(DataFrames_Cantidades)

Armamos un DataFrame para los migrantes recibidos.

In [397]:
# Inmigrantes = Migrantes[Migrantes['id_pais_destino'] == 'ARG'].copy()
# Inmigrantes.loc[:, 'emigrantes'] = 0
# Inmigrantes = Inmigrantes[['id_pais_origen', 'migrantes', 'emigrantes', 'anio']]
# Inmigrantes = Inmigrantes.rename(columns={'id_pais_origen': 'id_pais', 'migrantes': 'inmigrantes'})
# Inmigrantes

Armamos otro para los inmigrantes.

In [398]:
# Emigrantes = Migrantes[Migrantes['id_pais_origen'] == 'ARG'].copy()
# Emigrantes.loc[:, 'inmigrantes'] = 0
# Emigrantes = Emigrantes[['id_pais_destino', 'inmigrantes', 'migrantes', 'anio']]
# Emigrantes = Emigrantes.rename(columns={'id_pais_destino': 'id_pais', 'migrantes': 'emigrantes'})
# Emigrantes

Combinamos los datos.

In [399]:
# Migrantes = pd.DataFrame()
# ID_Pais_Lista = []
# Inmigrantes_Lista = []
# Emigrantes_Lista = []
# Anio_Lista = []

# for Index, Row in Inmigrantes.iterrows():
#     Inmigrantes['inmigrantes'][Index] = 

Guardamos el archivo.

In [400]:
Migrantes.to_csv('Tablas/TABLA. Migrantes.csv', index=False)

Migrantes.head(10)

,id_pais_origen,id_pais_destino,migrantes,anio
0,AFG,ARG,6.0,1960
1,ALB,ARG,4.0,1960
2,DZA,ARG,71.0,1960
3,AGO,ARG,786.0,1960
4,ARG,DZA,3.0,1960
5,ARG,AND,4.0,1960
6,ARG,AGO,6.0,1960
7,ARG,ATG,3.0,1960
8,ARG,ABW,8.0,1960
9,ARG,AUS,144.0,1960


### __Regiones__

***

Las regiones se extraen del dataset "Sedes".

In [401]:
Dataset_Sedes = 'Sedes'
Sedes_DB = pd.read_csv(Ruta + Dataset_Sedes + '.csv')

Arreglamos discordancias de datos entre los códigos de países en Sedes y Migraciones.

In [402]:
Sedes_DB['pais_iso_3'] = Sedes_DB['pais_iso_3'].replace('GRB', 'GBR')

Nos quedamos con las columna que nos interesa: region_geografica.

In [403]:
Regiones = pd.DataFrame(Sedes_DB['region_geografica'])

Eliminamos espacios adicionales.

In [404]:
Regiones['region_geografica'] = Regiones['region_geografica'].str.strip().str.replace(r'\s+', ' ', regex=True)

Nos quedamos con los valores únicos.

In [405]:
Regiones = Regiones.drop_duplicates()

Las ordenamos alfabéticamente y reseteamos el índice para no generar problemas.

In [406]:
Regiones = Regiones.sort_values(by='region_geografica').reset_index(drop=True)

Generamos una nueva columna: id, con números identificatorios.

In [407]:
Regiones['id'] = [x for x in range(1,len(Regiones)+1)]

Cambiamos los nombres de los atributos y ordenamos las columnas

In [408]:
Regiones = Regiones.rename(columns = {'region_geografica': 'nombre'})
Regiones = Regiones[['id', 'nombre']]

Chequeo final de nulos en la columna "id". Los eliminamos.

In [409]:
Regiones = Regiones.dropna(subset='id')

Guardamos el documento en un .csv.

In [410]:
Regiones.to_csv('Tablas/TABLA. Regiones.csv', index=False)

Regiones

,id,nombre
0,1,AMÉRICA CENTRAL Y CARIBE
1,2,AMÉRICA DEL NORTE
2,3,AMÉRICA DEL SUR
3,4,ASIA
4,5,EUROPA CENTRAL Y ORIENTAL
5,6,EUROPA OCCIDENTAL
6,7,OCEANÍA
7,8,ÁFRICA DEL NORTE Y CERCANO ORIENTE
8,9,ÁFRICA SUBSAHARIANA


### __Países__

***

Usamos los códigos y nombres de la base de datos "Migraciones.csv", ya que son más exhaustivos que los de las otras tablas. El procedimiento para encontrar los valores únicos es:

1. Armar tres DataFrames: uno con los países de origen -código y nombre-, otro con los países de destino -código y nombre- y otro con los países de "Sedes". De esa forma obtenemos todos los países que están en una u otra columna pero no en las otras.
2. Concatenarlos verticalmente, llamando las columnas "id" y "nombre". Este DataFrame se llama "Países".
3. Se eliminan las filas con valores duplicados en la columna "id". 

In [411]:
Paises_Origen = Migraciones_DB[['Country Origin Code', 'Country Origin Name']]
Paises_Destino = Migraciones_DB[['Country Dest Code', 'Country Dest Name']]
Paises_Sedes = Sedes_DB[['pais_iso_3', 'pais_castellano']]

Renombramos las columnas para que se llamen igual.

In [412]:
Nuevos_Nombres_Origen = {'Country Origin Code': 'id',
                         'Country Origin Name': 'nombre'}

Nuevos_Nombres_Destino = {'Country Dest Code': 'id',
                          'Country Dest Name': 'nombre'}

Nuevos_Nombres_Sedes = {'pais_iso_3': 'id',
                        'pais_castellano': 'nombre'}

Paises_Origen = Paises_Origen.rename(columns = Nuevos_Nombres_Origen)
Paises_Destino = Paises_Destino.rename(columns = Nuevos_Nombres_Destino)
Paises_Sedes = Paises_Sedes.rename(columns = Nuevos_Nombres_Sedes)

Concatenamos verticalmente los dos DataFrames y creamos "Paises".

In [413]:
Paises = pd.concat([Paises_Origen, Paises_Destino, Paises_Sedes], axis=0)

Eliminamos espacios adicionales.

In [414]:
for Columna in Paises.columns:
    Paises[Columna] = Paises[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)

Filtramos los valores repetidos de las columna "id". Además, reseteamos el índice para no generar problemas.

In [415]:
Paises = Paises.drop_duplicates(subset=['id']).reset_index(drop=True)

Eliminamos los duplicados en la columna "pais_iso_3" en el DataFrame "Sedes". Esto es para que no se generen filas de más en el join.

In [416]:
Paises_En_Sedes = Sedes_DB.drop_duplicates(subset='pais_iso_3')

Además, eliminamos espacios adicionales.

In [417]:
for Columna in ['region_geografica', 'pais_iso_3']:
    Paises_En_Sedes[Columna] = Paises_En_Sedes[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)

C:\Users\tomas\AppData\Local\Temp\ipykernel_5924\898895942.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Paises_En_Sedes[Columna] = Paises_En_Sedes[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)


Agregamos el 'id_region' correspondiente a cada país. Este se extrae del correspondiente al país en el dataset "Sedes". Si no se encuentra, queda NULL.

In [418]:
Paises = pd.merge(Paises, Paises_En_Sedes, left_on='id', right_on='pais_iso_3', how='left')[['id', 'nombre', 'region_geografica']]

Hacemos un LEFT_JOIN con el DataFrame "Regiones" para extraer los id de las regiones correspondientes.

In [419]:
Paises = pd.merge(Paises, Regiones, left_on='region_geografica', right_on='nombre', how='left')

Filtramos las columnas a las que necesitamos y cambiamos nombres.

In [420]:
Paises = Paises[['id_x', 'nombre_x', 'id_y']]

Paises = Paises.rename(columns={'id_x': 'id', 'nombre_x': 'nombre', 'id_y': 'id_region'})

Contamos los NULLS y los reemplazamos con "-" (esto es opcional).

NOTA: Este punto es problemático porque hay demasiados NULLS.

In [421]:
Paises['id_region'] = Paises['id_region'].fillna('-')

Ordenamos alfabéticamente por id.

In [422]:
Paises = Paises.sort_values(by='id').reset_index(drop=True)

Chequeo final de nulos en la columna "id". Los eliminamos.

In [423]:
Paises = Paises.dropna(subset='id')

Guardamos el documento.

In [424]:
Paises.to_csv('Tablas/TABLA. Paises.csv', index=False)

Paises

,id,nombre,id_region
0,ABW,Aruba,-
1,AFG,Afghanistan,-
2,AGO,Angola,9.0
3,AIA,Anguilla,-
4,ALB,Albania,-
...,...,...,...
228,YEM,"Yemen, Rep.",-
229,ZAF,South Africa,9.0
230,ZMB,Zambia,-
231,ZWE,Zimbabwe,-


### __Sedes__

***

Se extraen del dataset "Sedes". Nos quedamos con las columnas que nos interesan y conforman los atributos.

In [425]:
Sedes = Sedes_DB[['sede_id', 'sede_desc_castellano', 'pais_iso_3']]

Cambiamos los nombres de las columnas.

In [426]:
Sedes = Sedes.rename(columns={'sede_id': 'id', 'pais_iso_3': 'id_pais', 'sede_desc_castellano': 'nombre'})

Borramos espacios adicionales.

In [427]:
for Columna in Sedes.columns:
    Sedes[Columna] = Sedes[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)

Ordenamos alfabéticamente por id.

In [428]:
Sedes = Sedes.sort_values(by='id').reset_index(drop=True)

Chequeo final de nulos en la columna "id". Los eliminamos.

In [429]:
Sedes = Sedes.dropna(subset='id')

Guardamos el documento.

In [430]:
Sedes.to_csv('Tablas/TABLA. Sedes.csv', index=False)

Sedes

,id,nombre,id_pais
0,CALEG,Consulado General en Porto Alegre,BRA
1,CANTO,Consulado en Antofagasta,CHL
2,CAREN,Consulado General en Punta Arenas,CHL
3,CASUN,Consulado General en Asunción,PRY
4,CATLA,Consulado General en Atlanta,USA
...,...,...,...
159,EURUG,Embajada en Uruguay,URY
160,EVENE,Embajada en Venezuela,VEN
161,EVIET,Embajada en Vietnam,VNM
162,OCCAT,Oficina Comercial y Cultural Argentina en Taiwán,CHN


### __Secciones__

***

Para armar esta tabla utilizamos el dataset "Secciones.csv".

In [431]:
Dataset_Secciones = 'Secciones'
Secciones_DB = pd.read_csv(Ruta + Dataset_Secciones + '.csv')

Quitamos las columnas que no sirven y cambiamos los nombres de los atributos.

In [432]:
Secciones = Secciones_DB[['sede_id', 'sede_desc_castellano']]
Secciones = Secciones.rename(columns={'sede_id': 'id_sede', 'sede_desc_castellano': 'nombre'})

Borramos espacios adicionales.

In [433]:
for Columna in Secciones.columns:
    Secciones[Columna] = Secciones[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)

Ordenamos alfabéticamente por id.

In [434]:
Secciones = Secciones.sort_values(by='id_sede').reset_index(drop=True)

Chequeo final de nulos en la columna "id". Los eliminamos.

In [435]:
Secciones = Secciones.dropna(subset='id_sede')

Guardamos el documento.

In [436]:
Secciones.to_csv('Tablas/TABLA. Secciones.csv', index=False)

Secciones

,id_sede,nombre
0,CBARC,"Sección Cultura, Educación y Redes Sociales"
1,CBARC,Sección Administrativa
2,CBARC,Sección Consular (Asistencia al ciudadano/Visas)
3,CBARC,Sección Consular (Documentación/Asuntos previs...
4,CBARC,"Sección Comercial, Turismo e Inversiones"
...,...,...
510,EVENE,Sección Política
511,EVIET,Sección Prensa y Cultura
512,EVIET,Sección Política y Cooperación
513,EVIET,"Sección Económica, Comercial y de Inversiones"


### __Atributos foráneos__

***

Vamos a chequear que todos los valores correspondientes a las columnas que son atributos foráneos existan en la columna original correspondiente. Si no existe, se agrega ese valor en el DataFrame del que se extrae el dato.

##### Migraciones

En "Migrantes" hay dos atributos foráneos: "id_pais_origen" e "id_pais_destino", que provienen de "id" de la tabla "Paises". Al haberse armado la tabla "Paises", y, en particular su columna "id", a partir de las columnas "id_pais_destino" e "id_pais_origen", así como de "pais_iso_3" de la tabla "Sedes", no es necesario verificar, ya que todos los valores provenientes de estas columnas están en la columna "id".

##### Regiones

No tiene atributos foráneos.

##### Paises

Tiene un atributo foráneo: "id_region", proveniente de la columna "id" de la tabla Regiones. El "id" es un valor ficticio que agregamos para no escribir el nombre entero en toda la tabla repetidas veces. 